In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('moviedata.csv')

In [ ]:
df.head()

,Lead,Genre,Plot
0,Vijay,"Action, Crime, Drama, Thriller",An alcoholic professor is enrolled to teach at...
1,Vijay,"Action, Thriller","After years of successful operations, an elite..."
2,Vijay,"Action, Crime, Drama, Thriller",DCP Vijaya Kumar goes into hiding to bring up ...
3,Vijay,"Action, Drama, Political Drama",An NRI businessman returns home and learns abo...
4,Vijay,"Action, Crime, Drama, Thriller, Gangster",Things start to take an awry turn for a mild-m...


In [ ]:
df['prompt'] = "Suggest a plot for " + df['Genre'] + " movie starring " + df['Lead']
df['response'] = df['Plot']
df = df[['prompt', 'response']]

In [ ]:
df.shape

(100, 2)

In [ ]:
!pip install torch transformers datasets tokenizers

In [ ]:
import pandas as pddf["text"] = "Prompt: " + df["prompt"] + " Response: " + df["response"]


# Concatenate the prompt and response

train_texts = df["text"].tolist()

In [ ]:
from transformers import GPT2Tokenizer

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn’t have a padding token

# Tokenize dataset
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)

In [ ]:
import torch

class GPT2Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()  # Labels must be identical to input_ids
        return item

In [ ]:
train_dataset = GPT2Dataset(train_encodings)

In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    evaluation_strategy="no",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    report_to="none",  # Avoid unwanted logging issues
)


# Use Trainer API to train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=75, training_loss=1.7561075846354166, metrics={'train_runtime': 491.8799, 'train_samples_per_second': 0.61, 'train_steps_per_second': 0.152, 'total_flos': 14238374400000.0, 'train_loss': 1.7561075846354166, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")

('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

In [ ]:
from transformers import pipeline

# Load fine-tuned model
generator = pipeline("text-generation", model="./gpt2-finetuned", tokenizer=tokenizer)

# Generate response
prompt = "Prompt: Suggest a plot for a action, crime, drama movie starring Vijay Sethupathi Response: "
output = generator(prompt, max_length=200, num_return_sequences=1)

print(output[0]["generated_text"].replace(prompt, ""))

Device set to use cpu


 Thuduru, a young man, is caught in a corrupt scheme. However, he is rescued by his wife, Ravana, who convinces him to fight and reunite with her.
